In [1]:
import os
import csv
import cv2
from PIL import Image
from tqdm import tqdm
import numpy as np
from rembg import remove as remove_background


ASCII_CHARS = ["@", "#", "S", "%", "?", "*", "+", ";", ":", ",", "."]
ASCII_CHARS = [' ', '.', ',', ':', ';', '+', '*', '?', '%', 'S', '#', '@']

In [2]:
def resize_image(image, new_width=20):
    width, height = image.size
    ratio = height/width
    new_height = int(new_width * ratio)
    resized_image = image.resize((new_width, new_height)) # TODO copy
    return resized_image

def grayify(image):
    grayscale_image = image.convert("L")
    return grayscale_image

def pixels_to_ascii(image):
    pixels = image.getdata()
    duplicated_pixels = [pixel for pixel in pixels for _ in range(2)]
    characters = "".join([ASCII_CHARS[pixel//25] for pixel in duplicated_pixels])
    return characters

def sentencify(ascii_art_list: list, new_width):
    pixel_count = len(ascii_art_list) * 2
    ascii_image = "\n".join([ascii_art_list[index:(index+new_width)] for index in range(0, pixel_count, new_width)])
    return ascii_image

def csv_save(file_name, data):
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [3]:
def main(data_path, csv_ascii_20_path, csv_asccii_100_path):
    ascii_20, ascii_100 = [], []
    for datasets_name in os.listdir(data_path):
        dataset_path = os.path.join(data_path, datasets_name)

        for class_ in os.listdir(dataset_path):
            class_path = os.path.join(dataset_path, class_)

            for sample in tqdm(os.listdir(class_path)):
                sample_path = os.path.join(class_path, sample)
                #print(f"Processing: Dataset {datasets_name}, Class {class_}, Sample {sample}")
                try:
                    input = cv2.imread(sample_path)
                    image = remove_background(input)

                    img_blurred = cv2.GaussianBlur(image,(5,5),15)
                    gray_img = cv2.cvtColor(img_blurred , cv2.COLOR_BGR2GRAY)
                    img_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 5)

                    # To PIL Image
                    image = Image.fromarray(img_thresh)
                except:
                    print(sample_path, " is not a valid pathname to an image.")
                    continue

                image_grey_20 = grayify(resize_image(image.copy(), 20))
                image_grey_100 = grayify(resize_image(image.copy(), 100))
                ascii_20.append([sentencify(pixels_to_ascii(image_grey_20), 40), class_, sample])
                ascii_100.append([sentencify(pixels_to_ascii(image_grey_100), 200), class_, sample])

            csv_save(os.path.join(csv_ascii_20_path, f"_{datasets_name}_{class_}"), ascii_20)
            csv_save(os.path.join(csv_asccii_100_path, f"_{datasets_name}_{class_}"), ascii_100)
    print('===> Done <===')

In [4]:
data_path = os.path.join(os.getcwd(), "data")
data_path

'/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data'

In [5]:
ascii_20_path = os.path.join(os.getcwd(), "ascii_20", "ascii_20.csv")
ascii_20_path

'/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/ascii_20/ascii_20.csv'

In [6]:
ascii_100_path = os.path.join(os.getcwd(), "ascii_100", "ascii_100.csv")
ascii_100_path

'/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/ascii_100/ascii_100.csv'

In [7]:
main(data_path, ascii_20_path, ascii_100_path)

 24%|██▍       | 122/500 [01:33<04:34,  1.38it/s]

In [ ]:
import pandas as pd

df_ascii_20 = pd.read_csv(ascii_20_path, header=None, names=["ascii_art", "class", "sample_name"])
df_ascii_100 = pd.read_csv(ascii_100_path, header=None, names=["ascii_art", "class", "sample_name"])


In [ ]:
img = Image.open('/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data/FirstDataset/Cat/4.png')
pil_gray_image = img.convert('L')

In [ ]:
%time remove_background(img)

In [ ]:
%time remove_background(pil_gray_image)

In [ ]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt


path1 = '/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data/FirstDataset/Cat/1.png'
path2 = '/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data/FirstDataset/Cat/t90.png'
path3 = '/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data/FirstDataset/Cat/t91.png'
path4 = '/home/alex/Desktop/mistral_hackathon/src/image_to_ascii/data/FirstDataset/Cat/t83.png'

def test(path):
    pil_image = Image.open(path)

    img = np.array(pil_image)
    img = remove_background(img)

    img_blurred = cv2.GaussianBlur(img,(5,5),15)
    gray_img = cv2.cvtColor(img_blurred , cv2.COLOR_BGR2GRAY)
    img_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 5)
    pil_image_processed = Image.fromarray(img_thresh)

    plt.imshow(pil_image_processed, cmap='gray')
    plt.show()

test(path1)
test(path2)
test(path3)
test(path4)